In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')


In [ ]:
train_df.count(), train_df.dtypes


In [ ]:
train_df.target.value_counts()


In [ ]:
fs = [f'f_{i:02d}' for i in range(0, 31)]
num_df = train_df[['target']+fs].drop(['f_27'], axis=1)


In [ ]:
num_df.corr()


In [ ]:
display(train_df['f_27'])
print('% of unique sequence in train', len(train_df['f_27'].unique()) / len(train_df['f_27']))
print('% of unique sequence in test', len(test_df['f_27'].unique()) / len(test_df['f_27']))


In [ ]:
f27_target_seq_counts = train_df[['target', 'f_27']].groupby(['target', 'f_27']).size().unstack(fill_value=0).T

most_occur_in_0 = f27_target_seq_counts.sort_values(by=[0], ascending=False)
most_occur_in_1 = f27_target_seq_counts.sort_values(by=[1], ascending=False)

fig, axs = plt.subplots(2, figsize=(20,10))
most_occur_in_0.head(5000).plot(kind='area', stacked=False, ax=axs[0], title='Top 5000 occuring duplicated sequences for target 0')
most_occur_in_1.head(5000).plot(kind='area', stacked=False, ax=axs[1], title='Top 5000 occuring duplicated sequences for target 1')
fig.tight_layout()


In [ ]:
f27_map = most_occur_in_0.reset_index()
f27_map.columns = ['f_27', 'f_27_tar0', 'f_27_tar1']
f27_map


In [ ]:
print('Among all sequences in test set')
print('% of same duplicated sequence as in train set:', test_df['f_27'].isin(f27_target_seq_counts.index).sum() / len(test_df))
print('# of same top 100 duplicated sequence as in train set, for target 0:',test_df['f_27'].isin(most_occur_in_0.index).sum())
print('# of same top 100 duplicated sequence as in train set, for target 1:', test_df['f_27'].isin(most_occur_in_1.index).sum())


In [ ]:
# Splitting the sequences into single letters
f27_split = train_df['f_27'].str.split(pat ="\s*", expand = True).iloc[:,1:-1]
f27_split_test = test_df['f_27'].str.split(pat ="\s*", expand = True).iloc[:,1:-1]


In [ ]:
fig, axs = plt.subplots(10, figsize=(10,30))
for i in range(10):
    f27_split.iloc[:, i].value_counts().plot(kind='bar', ax=axs[i], title=f'sequence position {i}')
fig.tight_layout()


In [ ]:
f27_split['target'] = train_df['target']


In [ ]:
fig, axs = plt.subplots(10, figsize=(10,30))
for i in range(10):
    target_seqpos_counts = f27_split[['target', i+1]].groupby(['target', i+1]).size().unstack(fill_value=0).T
    target_seqpos_counts.plot(kind='bar', ax=axs[i], title=f'sequence position {i}')
fig.tight_layout()


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
f27_encoded = enc.fit_transform(f27_split.drop(['target'], axis=1)).astype(int)
f27_encoded_test = enc.transform(f27_split_test).astype(int)
f27_encoded[:3]


In [ ]:
f27_split.drop(['target'], axis=1).head(3)


In [ ]:
f27_split_df = pd.DataFrame(f27_encoded)
f27_split_df.columns = [f'f_27_pos{i}' for i in range(10)]
f27_split_test_df = pd.DataFrame(f27_encoded_test)
f27_split_test_df.columns = [f'f_27_pos{i}' for i in range(10)]


In [ ]:
full_df = pd.concat([train_df, f27_split_df], axis=1)
full_df["f_27_unique"] = full_df["f_27"].apply(lambda x: len(set(x)))

full_test_df = pd.concat([test_df, f27_split_test_df], axis=1)
full_test_df["f_27_unique"] = full_test_df["f_27"].apply(lambda x: len(set(x)))


In [ ]:
# full_df = full_df.merge(f27_map, how='left', on='f_27')

# full_test_df = full_test_df.merge(f27_map, how='left', on='f_27').fillna(0)
# full_test_df[['f_27_tar0', 'f_27_tar1']] = full_test_df[['f_27_tar0', 'f_27_tar1']].astype(int)


In [ ]:
# temp = full_test_df[[f'f_27_pos{i}' for i in range(10)] + ['f_27_unique'] + ['f_27_tar0', 'f_27_tar1']]
# temp


In [ ]:
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=2)
# imputer.fit_transform(temp.to_numpy())
# full_test_df


In [ ]:
X_train = full_df.drop(['id', 'target'], axis=1)
X_train['f_27'] = X_train['f_27'].astype('category')
y_train = train_df.target

X_test = full_test_df.drop(['id'], axis=1)
X_test['f_27'] = X_test['f_27'].astype('category')


In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.layers.experimental import preprocessing
# from sklearn.model_selection import train_test_split, StratifiedKFold
# from sklearn.metrics import r2_score

# model = tf.keras.Sequential([
#     layers.LSTM(32, input_shape=X_shape, return_sequences=True),
#     layers.LSTM(32, input_shape=(X_shape[0], 32), return_sequences=False),
#     layers.Dense(32, activation='relu', input_shape=X_shape),
#     layers.Dense(16, activation='relu'),
#     layers.Dense(1, activation='sigmoid')
# ])

# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(0.3, decay_steps=200, decay_rate=0.85, staircase=False)
# model.summary()
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
#               metrics=['accuracy', tf.keras.metrics.AUC()], 
#               optimizer=tf.keras.optimizers.Adam(lr_schedule))


In [ ]:
# Xt, Xv, yt, yv = train_test_split(X_train, y_train, test_size=0.3)
# bs = len(Xt) // 500
# bs


In [ ]:
# history = model.fit(Xt, yt, batch_size=bs, validation_data=(Xv, yv), verbose=1, epochs=10)


In [ ]:
# model.predict(Xv[:10])


In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
param = {'objective': 'cross_entropy', 'metric': ['auc', 'binary_logloss']}
# num_round = 100
# hist = lgb.cv(param, train_data, num_round, nfold=5)


In [ ]:
# hist['auc-mean'][-1]


In [ ]:
clf = lgb.train(param, train_data)


In [ ]:
y_pred = clf.predict(X_test)


In [ ]:
submission = test_df[['id']]
submission['target'] = y_pred
submission.to_csv('submission.csv', index=False)


In [ ]:
pd.read_csv('submission.csv')
